<a href="https://colab.research.google.com/github/markleschinsky/fed-travel-rates/blob/main/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Querying Zip Codes from U.S. Census

In [13]:
import requests
import pandas as pd

def fetch_zcta_population():
    url = "https://api.census.gov/data/2023/acs/acs5?get=NAME,B01001_001E&for=zip+code+tabulation+area:*"
    response = requests.get(url)
    data = response.json()

    # Convert data to DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])

    # Convert population to numeric and sort by population
    df['B01001_001E'] = pd.to_numeric(df['B01001_001E'])
    df_sorted = df.sort_values(by='B01001_001E', ascending=False)

    # Rename columns for clarity
    df_sorted.rename(columns={
        'NAME': 'Zip Code Area',
        'B01001_001E': 'Population Size'
    }, inplace=True)

    # Extract zip code from 'Zip Code Area'
    df_sorted['Zip Code'] = df_sorted['Zip Code Area'].apply(lambda x: x.split(' ')[-1])

    return df_sorted

# Fetch the DataFrame
top_zctas = fetch_zcta_population()

# Drop all columns except for Population Size and Zip Code
top_zctas_filtered = top_zctas[['Population Size', 'Zip Code']]

# Display the updated DataFrame
top_zctas_filtered.head()

top_1000_zips = top_zctas_filtered
top_1000_zips = top_1000_zips.head(1000)

top_1000_zips.head()

,Population Size,Zip Code
27396,137213,77494
2495,136784,08701
27359,123042,77449
27894,118437,78660
27345,112211,77433


### Importing State Data by Zip Code

In [9]:
df_zip_states = pd.read_csv("uszips.csv")
df_zip_states["zip"] = df_zip_states["zip"].astype(str).str.zfill(5)
df_zip_states.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,00601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,16721.0,100.2,72001,Adjuntas,"{""72001"": 98.74, ""72141"": 1.26}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,00602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37510.0,477.6,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
2,00603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,48317.0,543.1,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
3,00606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,5435.0,47.3,72093,Maricao,"{""72093"": 82.27, ""72153"": 11.66, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,00610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,25413.0,264.4,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.82, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico


### Merging State Data to Zip Codes

In [14]:
df_zips_and_states = top_1000_zips.merge(right=df_zip_states[["zip", "state_id", "lat", "lng", "city", "density", "county_fips", "county_name"]], left_on="Zip Code", right_on="zip")

df_zips_and_states = df_zips_and_states[df_zips_and_states["Population Size"] != 0]
df_zips_and_states = df_zips_and_states[["Population Size", "Zip Code", "state_id", "lat", "lng", "city", "density", "county_fips", "county_name"]]

df_zips_and_states.columns = ['Population Size', 'Zip Code', 'State', 'Latitude', 'Longitude', 'City', 'Population Density', 'County FIPS', 'County Name']

df_zips_and_states.head()

,Population Size,Zip Code,State,Latitude,Longitude,City,Population Density,County FIPS,County Name
0,137213,77494,TX,29.74566,-95.82302,Katy,1398.4,48157,Fort Bend
1,136784,08701,NJ,40.07635,-74.20311,Lakewood,2135.0,34029,Ocean
2,123042,77449,TX,29.83674,-95.73547,Katy,1756.9,48201,Harris
3,118437,78660,TX,30.44361,-97.59558,Pflugerville,1025.1,48453,Travis
4,112211,77433,TX,29.94920,-95.73979,Cypress,734.7,48201,Harris


### Convert Zip Codes to List for Subsequent API Querying

In [15]:
zip_codes = df_zips_and_states["Zip Code"].tolist()
zip_codes = zip_codes[:1000]
zip_codes[:10]

['77494',
 '08701',
 '77449',
 '78660',
 '77433',
 '60629',
 '77084',
 '11368',
 '11208',
 '79936']

### Query Federal Government GSA API for Per Diem Rates

In [20]:
import requests
import pandas as pd
import time

headers = {
    "x-api-key": "QMVs4H9JiK1KF23lDzesf1GNbHIJ3IhaSraQxU5r",
}

# Year for which to retrieve per diem rates
year = "2024"

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each zip code
for zip_code in zip_codes[:10]:
    url = f"https://api.gsa.gov/travel/perdiem/v2/rates/zip/{zip_code}/year/{year}"

    try:
        # Send GET request
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse JSON response
            data = response.json()

            # Extract relevant data for DataFrame
            rates = data.get('rates', [])
            if rates:  # Ensure rates exist
                for rate in rates:
                    for rate_detail in rate.get('rate', []):
                        months = rate_detail.get('months', {}).get('month', [])
                        for month in months:
                            # Create a DataFrame row for each month
                            row = {
                                'Zip Code': rate_detail.get('zip'),
                                'City': rate_detail.get('city'),
                                'County': rate_detail.get('county'),
                                'State': rate.get('state'),
                                'Year': rate.get('year'),
                                'Month': month.get('long'),
                                'Rate': month.get('value'),
                                'Meals': rate_detail.get('meals')
                            }
                            # Append the row to a DataFrame
                            dataframes.append(pd.DataFrame([row]))
            time.sleep(0.1)  # Add a delay of 0.1 seconds between requests
        else:
            print(f"Failed to retrieve data for {zip_code}. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred for {zip_code}: {e}")

# Combine all DataFrames into a single DataFrame
if dataframes:  # Ensure there are DataFrames to concatenate
    gsa_rates = pd.concat(dataframes, ignore_index=True)
    # Display the combined DataFrame
    print(gsa_rates.head())
else:
    print("No data available to concatenate.")

  Zip Code           City County State  Year     Month  Rate  Meals
0    77494  Standard Rate   None    TX  2024   January   107     59
1    77494  Standard Rate   None    TX  2024  February   107     59
2    77494  Standard Rate   None    TX  2024     March   107     59
3    77494  Standard Rate   None    TX  2024     April   107     59
4    77494  Standard Rate   None    TX  2024       May   107     59


### Zip Codes and Per Diem Rates for Each Month

In [22]:
gsa_rates.head()

,Zip Code,City,County,State,Year,Month,Rate,Meals
0,77494,Standard Rate,None,TX,2024,January,107,59
1,77494,Standard Rate,None,TX,2024,February,107,59
2,77494,Standard Rate,None,TX,2024,March,107,59
3,77494,Standard Rate,None,TX,2024,April,107,59
4,77494,Standard Rate,None,TX,2024,May,107,59


### Zip Codes and Averaged Per Diem Rates

In [26]:
# calculate average per diem rate for each zip code
per_diem = gsa_rates.groupby('Zip Code')['Rate'].mean()
per_diem = pd.DataFrame(per_diem)

# Extract unique city names associated with each zip code
unique_cities = gsa_rates[['Zip Code', 'City']].drop_duplicates()

# If there are multiple cities per zip code, take the first one
city_by_zip = unique_cities.groupby('Zip Code')['City'].first()

# Add the City column to the per_diem DataFrame
per_diem['Zone'] = per_diem.index.map(city_by_zip)

per_diem.head()

,Rate,Zone
Zip Code,,
11208,256.666667,New York City
11368,256.666667,New York City
60629,196.416667,Chicago
77084,122.000000,Houston
77433,122.000000,Houston


### Zip Codes and Averaged Meal Rates

In [27]:
meals = gsa_rates.groupby('Zip Code')['Meals'].mean()

meals = pd.DataFrame(meals)

meals.head()

,Meals
Zip Code,
11208,79.0
11368,79.0
60629,79.0
77084,69.0
77433,69.0


### Import House Prices by Zip Code Nationwide

In [29]:
df_house_prices=pd.read_csv("zillow.csv")

df_house_prices['Zip Code'] = df_house_prices['RegionName'].astype(str).str.zfill(5)

df_house_prices.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,Zip Code
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,209050.476760,...,495075.413843,495380.402566,495002.983332,495408.561941,496096.305465,497215.161891,497931.319966,498348.024838,498770.070517,77494
1,61148,2,8701,zip,NJ,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County,129618.780605,...,574111.688523,579228.585853,583825.798950,588765.877710,594583.485735,599788.275075,603106.031473,605075.749335,605701.713141,08701
2,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,103655.528341,...,282147.022097,282107.280779,281872.890421,281691.186379,281502.991848,281086.410278,280298.902221,279570.221742,278960.087685,77449
3,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,146323.411563,...,452805.954270,453500.452732,452997.379756,452371.998659,453320.495788,453075.343776,451863.571974,449627.300182,447532.075931,11368
4,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,102106.861831,...,274613.196678,274446.918170,274155.370186,273883.459171,273631.254558,273125.765642,272528.815424,272190.921082,271949.294782,77084


### Merge Zip Codes, Census Data, Per Diem Data, and House Price Data

In [34]:
# merge house prices with per diem rates on the zip code
diem_and_house_price = per_diem.merge(right=df_house_prices[["2020-06-30", "Zip Code"]], left_on="Zip Code", right_on="Zip Code")

# rename columns
diem_and_house_price.columns = ["Zip Code", "Per Diem Daily Rate", "Zone", "House Price"]

# merge and add additional info from U.S. Census Bureau for each zip code
dataset = diem_and_house_price.merge(right=df_zips_and_states[["Zip Code", "City", "State", "Population Size", "Population Density", "County FIPS", "County Name"]], left_on="Zip Code", right_on="Zip Code")

# rename columns
dataset.columns = ["Zip Code", "Per Diem Daily Rate", "Rate Zone", "House Price", "City", "State", "Population Size", "Population Density", "County FIPS", "County Name"]

# create column to track if the rate is designated as a "Standard Rate" by the GSA
dataset["isStandard"] = dataset["Rate Zone"] == "Standard Rate"

dataset.head()

,Zip Code,Per Diem Daily Rate,Rate Zone,House Price,City,State,Population Size,Population Density,County FIPS,County Name,isStandard
0,11208,256.666667,New York City,594009.788749,Brooklyn,NY,106901,15421.6,36047,Kings,False
1,11368,256.666667,New York City,511254.023443,Corona,NY,107060,15695.6,36081,Queens,False
2,60629,196.416667,Chicago,188064.568952,Chicago,IL,109292,5986.9,17031,Cook,False
3,77084,122.000000,Houston,191487.099173,Houston,TX,109115,1327.5,48201,Harris,False
4,77433,122.000000,Houston,299719.628351,Cypress,TX,112211,734.7,48201,Harris,False
